In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import seaborn as sns

import tqdm
from nba_api.stats.endpoints.shotchartdetail import ShotChartDetail
import requests
from nba_api.stats.static.players import find_players_by_full_name
from nba_api.stats.static.teams import find_teams_by_full_name
from adjustText import adjust_text
from io import BytesIO
import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
import time
from PIL import Image
from collections import defaultdict

from math import cos, sin, radians, sqrt, tan, atan, degrees

In [2]:
from nba_api.stats.endpoints.playbyplayv2 import PlayByPlayV2

from nba_api.stats.endpoints.playbyplay import PlayByPlay
from nba_api.stats.endpoints.leaguegamefinder import LeagueGameFinder
from nba_api.stats.static.players import find_players_by_full_name
from nba_api.stats.static.teams import find_teams_by_full_name
from nba_api.stats.endpoints.playergamelogs import PlayerGameLogs
from nba_api.stats.endpoints.boxscoretraditionalv2 import BoxScoreTraditionalV2
from nba_api.stats.endpoints.shotchartdetail import ShotChartDetail

from nba_api.stats.endpoints.leaguedashteamshotlocations import LeagueDashTeamShotLocations
from nba_api.stats.endpoints.leaguedashplayerptshot import LeagueDashPlayerPtShot

from nba_api.stats.endpoints.synergyplaytypes import SynergyPlayTypes
from nba_api.stats.endpoints.leaguedashptstats import LeagueDashPtStats
from nba_api.stats.endpoints.leaguedashplayerstats import LeagueDashPlayerStats

from nba_api.stats.endpoints.leaguehustlestatsplayer import LeagueHustleStatsPlayer

In [3]:
player_stats = LeagueDashPlayerStats(season='2021-22').get_data_frames()[0]

In [4]:
player_stats_last = LeagueDashPlayerStats(season='2020-21').get_data_frames()[0]

In [5]:
current_shots = ShotChartDetail(team_id=0, player_id=0, season_nullable='2021-22', context_measure_simple='FGA', season_type_all_star='Regular Season', timeout=840)


In [6]:
current_df = current_shots.get_data_frames()[0]

In [7]:
last_shots = ShotChartDetail(team_id=0, player_id=0, season_nullable='2020-21', timeout=900, context_measure_simple='FGA', season_type_all_star='Regular Season')

In [8]:
last_df = last_shots.get_data_frames()[0]

In [9]:
current_df.head()

,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
0,Shot Chart Detail,0022100001,7,1628960,Grayson Allen,1610612749,Milwaukee Bucks,1,11,42,...,Left Side Center(LC),24+ ft.,26,-164,210,1,0,20211019,MIL,BKN
1,Shot Chart Detail,0022100001,15,203507,Giannis Antetokounmpo,1610612749,Milwaukee Bucks,1,11,13,...,Center(C),24+ ft.,26,0,263,1,0,20211019,MIL,BKN
2,Shot Chart Detail,0022100001,18,203507,Giannis Antetokounmpo,1610612749,Milwaukee Bucks,1,11,1,...,Center(C),8-16 ft.,14,5,147,1,0,20211019,MIL,BKN
3,Shot Chart Detail,0022100001,20,1629651,Nic Claxton,1610612751,Brooklyn Nets,1,10,47,...,Center(C),Less Than 8 ft.,0,1,4,1,1,20211019,MIL,BKN
4,Shot Chart Detail,0022100001,22,203507,Giannis Antetokounmpo,1610612749,Milwaukee Bucks,1,10,31,...,Center(C),8-16 ft.,10,15,104,1,0,20211019,MIL,BKN


In [10]:
current_df.iloc[0]

GRID_TYPE                 Shot Chart Detail
GAME_ID                          0022100001
GAME_EVENT_ID                             7
PLAYER_ID                           1628960
PLAYER_NAME                   Grayson Allen
TEAM_ID                          1610612749
TEAM_NAME                   Milwaukee Bucks
PERIOD                                    1
MINUTES_REMAINING                        11
SECONDS_REMAINING                        42
EVENT_TYPE                      Missed Shot
ACTION_TYPE                       Jump Shot
SHOT_TYPE                    3PT Field Goal
SHOT_ZONE_BASIC           Above the Break 3
SHOT_ZONE_AREA         Left Side Center(LC)
SHOT_ZONE_RANGE                     24+ ft.
SHOT_DISTANCE                            26
LOC_X                                  -164
LOC_Y                                   210
SHOT_ATTEMPTED_FLAG                       1
SHOT_MADE_FLAG                            0
GAME_DATE                          20211019
HTM                             

In [11]:
grouped_current = current_df.groupby(['PLAYER_ID', 'PLAYER_NAME', 'ACTION_TYPE','SHOT_ZONE_BASIC', 'SHOT_ZONE_AREA', 'SHOT_ZONE_RANGE']).agg(
    {
        'SHOT_MADE_FLAG': 'sum',
        'SHOT_ATTEMPTED_FLAG': 'sum'
    }
)

In [12]:
grouped_current = grouped_current.reset_index()

In [13]:
sorted(current_df.ACTION_TYPE.unique().tolist())

['Alley Oop Dunk Shot',
 'Alley Oop Layup shot',
 'Cutting Dunk Shot',
 'Cutting Finger Roll Layup Shot',
 'Cutting Layup Shot',
 'Driving Bank Hook Shot',
 'Driving Dunk Shot',
 'Driving Finger Roll Layup Shot',
 'Driving Floating Bank Jump Shot',
 'Driving Floating Jump Shot',
 'Driving Hook Shot',
 'Driving Layup Shot',
 'Driving Reverse Dunk Shot',
 'Driving Reverse Layup Shot',
 'Dunk Shot',
 'Fadeaway Jump Shot',
 'Finger Roll Layup Shot',
 'Floating Jump shot',
 'Hook Bank Shot',
 'Hook Shot',
 'Jump Bank Shot',
 'Jump Shot',
 'Layup Shot',
 'Pullup Jump shot',
 'Putback Dunk Shot',
 'Putback Layup Shot',
 'Reverse Dunk Shot',
 'Reverse Layup Shot',
 'Running Alley Oop Dunk Shot',
 'Running Alley Oop Layup Shot',
 'Running Dunk Shot',
 'Running Finger Roll Layup Shot',
 'Running Jump Shot',
 'Running Layup Shot',
 'Running Pull-Up Jump Shot',
 'Running Reverse Dunk Shot',
 'Running Reverse Layup Shot',
 'Step Back Bank Jump Shot',
 'Step Back Jump shot',
 'Tip Dunk Shot',
 'Tip 

## Shot Types

* Floaters

* Hook Shot

* Fadeaway Shot



### Hooks

In [14]:
hook_shots_only = current_df.loc[current_df.ACTION_TYPE.str.contains('Hook')]

In [15]:
hook_shots_agg = hook_shots_only.groupby(['PLAYER_ID', 'PLAYER_NAME']).agg(
    {
        'SHOT_MADE_FLAG': 'sum',
        'SHOT_ATTEMPTED_FLAG': 'sum'
    }
).reset_index()

In [16]:
hook_shots_agg = hook_shots_agg.sort_values(by='SHOT_ATTEMPTED_FLAG', ascending=False)

In [17]:
hook_shots_agg.loc[:, 'HOOK_PCT'] = hook_shots_agg.SHOT_MADE_FLAG / hook_shots_agg.SHOT_ATTEMPTED_FLAG * 100

In [18]:
hook_last = last_df.loc[last_df.ACTION_TYPE.str.contains('Hook')]
hook_last_agg = hook_last.groupby(['PLAYER_ID', 'PLAYER_NAME']).agg(
    {
        'SHOT_MADE_FLAG': 'sum',
        'SHOT_ATTEMPTED_FLAG': 'sum'
    }
).reset_index()

hook_last_agg = hook_last_agg.sort_values(by='SHOT_ATTEMPTED_FLAG', ascending=False)

In [19]:
hook_last_agg.loc[:, 'HOOK_PCT'] = hook_last_agg.SHOT_MADE_FLAG / hook_last_agg.SHOT_ATTEMPTED_FLAG * 100

In [20]:
merged_hook = pd.merge(left=hook_shots_agg, right=hook_last_agg, on=('PLAYER_ID', 'PLAYER_NAME'), suffixes=('_CUR', '_LAST'))

In [21]:
merged_hook = merged_hook.sort_values(by='SHOT_ATTEMPTED_FLAG_CUR', ascending=False)

In [22]:
names_games = player_stats.loc[:, ['PLAYER_ID', 'PLAYER_NAME', 'GP']]
names_games_last = player_stats_last.loc[:, ['PLAYER_ID', 'PLAYER_NAME', 'GP']]

In [23]:
merged_hook = pd.merge(left=merged_hook, right=names_games, on=['PLAYER_ID', 'PLAYER_NAME'])

In [24]:
merged_hook = pd.merge(left=merged_hook, right=names_games_last, on=['PLAYER_ID', 'PLAYER_NAME'], suffixes=('_CUR', '_LAST'))

In [25]:
merged_hook.loc[:, 'SHOT_MADE_FLAG_CUR_G'] = merged_hook.SHOT_MADE_FLAG_CUR / merged_hook.GP_CUR
merged_hook.loc[:, 'SHOT_ATTEMPTED_FLAG_CUR_G'] = merged_hook.SHOT_ATTEMPTED_FLAG_CUR / merged_hook.GP_CUR
merged_hook.loc[:, 'SHOT_MADE_FLAG_LAST_G'] = merged_hook.SHOT_MADE_FLAG_LAST / merged_hook.GP_LAST
merged_hook.loc[:, 'SHOT_ATTEMPTED_FLAG_LAST_G'] = merged_hook.SHOT_ATTEMPTED_FLAG_LAST / merged_hook.GP_LAST

In [39]:
merged_hook = merged_hook.loc[(merged_hook.GP_CUR > 15) & (merged_hook.SHOT_ATTEMPTED_FLAG_CUR > 30)]

In [40]:
merged_hook.loc[:, 'SHOT_ATTEMPTED_DIFF'] = merged_hook.SHOT_ATTEMPTED_FLAG_CUR_G - merged_hook.SHOT_ATTEMPTED_FLAG_LAST_G

In [41]:
merged_hook.loc[:, 'SHOT_PCT_DIFF'] = merged_hook.HOOK_PCT_CUR - merged_hook.HOOK_PCT_LAST

In [42]:
merged_hook.sort_values(by='HOOK_PCT_CUR', ascending=False)

,PLAYER_ID,PLAYER_NAME,SHOT_MADE_FLAG_CUR,SHOT_ATTEMPTED_FLAG_CUR,HOOK_PCT_CUR,SHOT_MADE_FLAG_LAST,SHOT_ATTEMPTED_FLAG_LAST,HOOK_PCT_LAST,GP_CUR,GP_LAST,SHOT_MADE_FLAG_CUR_G,SHOT_ATTEMPTED_FLAG_CUR_G,SHOT_MADE_FLAG_LAST_G,SHOT_ATTEMPTED_FLAG_LAST_G,SHOT_ATTEMPTED_DIFF,SHOT_PCT_DIFF
3,1629028,Deandre Ayton,93,141,65.957447,52,92,56.521739,58,69,1.603448,2.431034,0.753623,1.333333,1.097701,9.435708
44,1626158,Richaun Holmes,24,37,64.864865,24,45,53.333333,45,61,0.533333,0.822222,0.393443,0.737705,0.084517,11.531532
26,202355,Hassan Whiteside,29,45,64.444444,29,59,49.152542,65,36,0.446154,0.692308,0.805556,1.638889,-0.946581,15.291902
47,1630168,Onyeka Okongwu,23,36,63.888889,11,23,47.826087,48,50,0.479167,0.750000,0.220000,0.460000,0.290000,16.062802
27,203507,Giannis Antetokounmpo,28,45,62.222222,23,52,44.230769,67,61,0.417910,0.671642,0.377049,0.852459,-0.180817,17.991453
2,203999,Nikola Jokic,106,171,61.988304,113,176,64.204545,74,72,1.432432,2.310811,1.569444,2.444444,-0.133634,-2.216241
4,1626157,Karl-Anthony Towns,82,135,60.740741,48,94,51.063830,74,50,1.108108,1.824324,0.960000,1.880000,-0.055676,9.676911
46,1629002,Chimezie Metu,22,37,59.459459,11,19,57.894737,60,36,0.366667,0.616667,0.305556,0.527778,0.088889,1.564723
29,200782,P.J. Tucker,25,43,58.139535,2,2,100.000000,71,52,0.352113,0.605634,0.038462,0.038462,0.567172,-41.860465
30,203083,Andre Drummond,25,43,58.139535,49,115,42.608696,73,46,0.342466,0.589041,1.065217,2.500000,-1.910959,15.530839


In [30]:
merged_hook.columns

Index(['PLAYER_ID', 'PLAYER_NAME', 'SHOT_MADE_FLAG_CUR',
       'SHOT_ATTEMPTED_FLAG_CUR', 'HOOK_PCT_CUR', 'SHOT_MADE_FLAG_LAST',
       'SHOT_ATTEMPTED_FLAG_LAST', 'HOOK_PCT_LAST', 'GP_CUR', 'GP_LAST',
       'SHOT_MADE_FLAG_CUR_G', 'SHOT_ATTEMPTED_FLAG_CUR_G',
       'SHOT_MADE_FLAG_LAST_G', 'SHOT_ATTEMPTED_FLAG_LAST_G',
       'SHOT_ATTEMPTED_DIFF', 'SHOT_PCT_DIFF'],
      dtype='object')

### Floaters

In [46]:
target_shots_only = current_df.loc[current_df.ACTION_TYPE.str.contains('Floating')]

target_shots_agg = target_shots_only.groupby(['PLAYER_ID', 'PLAYER_NAME']).agg(
    {
        'SHOT_MADE_FLAG': 'sum',
        'SHOT_ATTEMPTED_FLAG': 'sum'
    }
).reset_index()

target_shots_agg = target_shots_agg.sort_values(by='SHOT_ATTEMPTED_FLAG', ascending=False)

target_shots_agg.loc[:, 'SHOT_PCT_CUR'] = target_shots_agg.SHOT_MADE_FLAG / target_shots_agg.SHOT_ATTEMPTED_FLAG * 100

In [47]:
target_last = last_df.loc[last_df.ACTION_TYPE.str.contains('Floating')]
target_last_agg = target_last.groupby(['PLAYER_ID', 'PLAYER_NAME']).agg(
    {
        'SHOT_MADE_FLAG': 'sum',
        'SHOT_ATTEMPTED_FLAG': 'sum'
    }
).reset_index()

target_last_agg = target_last_agg.sort_values(by='SHOT_ATTEMPTED_FLAG', ascending=False)

target_last_agg.loc[:, 'SHOT_PCT_LAST'] = target_last_agg.SHOT_MADE_FLAG / target_last_agg.SHOT_ATTEMPTED_FLAG * 100

In [48]:
merged_target = pd.merge(left=target_shots_agg, right=target_last_agg, on=('PLAYER_ID', 'PLAYER_NAME'), suffixes=('_CUR', '_LAST'))

merged_target = merged_target.sort_values(by='SHOT_ATTEMPTED_FLAG_CUR', ascending=False)

names_games = player_stats.loc[:, ['PLAYER_ID', 'PLAYER_NAME', 'GP']]
names_games_last = player_stats_last.loc[:, ['PLAYER_ID', 'PLAYER_NAME', 'GP']]

merged_target = pd.merge(left=merged_target, right=names_games, on=['PLAYER_ID', 'PLAYER_NAME'])

merged_target = pd.merge(left=merged_target, right=names_games_last, on=['PLAYER_ID', 'PLAYER_NAME'], suffixes=('_CUR', '_LAST'))

merged_target.loc[:, 'SHOT_MADE_FLAG_CUR_G'] = merged_target.SHOT_MADE_FLAG_CUR / merged_target.GP_CUR
merged_target.loc[:, 'SHOT_ATTEMPTED_FLAG_CUR_G'] = merged_target.SHOT_ATTEMPTED_FLAG_CUR / merged_target.GP_CUR
merged_target.loc[:, 'SHOT_MADE_FLAG_LAST_G'] = merged_target.SHOT_MADE_FLAG_LAST / merged_target.GP_LAST
merged_target.loc[:, 'SHOT_ATTEMPTED_FLAG_LAST_G'] = merged_target.SHOT_ATTEMPTED_FLAG_LAST / merged_target.GP_LAST

merged_target = merged_target.loc[(merged_target.GP_CUR > 15) & (merged_target.GP_LAST > 15) & (merged_target.SHOT_ATTEMPTED_FLAG_CUR > 50)]

merged_target.loc[:, 'SHOT_ATTEMPTED_DIFF'] = merged_target.SHOT_ATTEMPTED_FLAG_CUR_G - merged_target.SHOT_ATTEMPTED_FLAG_LAST_G

merged_target.loc[:, 'SHOT_PCT_DIFF'] = merged_target.SHOT_PCT_CUR - merged_target.SHOT_PCT_LAST

In [49]:
merged_target.loc[merged_target.SHOT_ATTEMPTED_FLAG_CUR > 1].sort_values(by='SHOT_ATTEMPTED_FLAG_CUR', ascending=False)

,PLAYER_ID,PLAYER_NAME,SHOT_MADE_FLAG_CUR,SHOT_ATTEMPTED_FLAG_CUR,SHOT_PCT_CUR,SHOT_MADE_FLAG_LAST,SHOT_ATTEMPTED_FLAG_LAST,SHOT_PCT_LAST,GP_CUR,GP_LAST,SHOT_MADE_FLAG_CUR_G,SHOT_ATTEMPTED_FLAG_CUR_G,SHOT_MADE_FLAG_LAST_G,SHOT_ATTEMPTED_FLAG_LAST_G,SHOT_ATTEMPTED_DIFF,SHOT_PCT_DIFF
0,1629027,Trae Young,176,369,47.696477,175,372,47.043011,76,63,2.315789,4.855263,2.777778,5.904762,-1.049499,0.653466
1,1629029,Luka Doncic,134,265,50.566038,125,222,56.306306,65,66,2.061538,4.076923,1.893939,3.363636,0.713287,-5.740269
2,1629630,Ja Morant,115,264,43.560606,99,237,41.772152,57,63,2.017544,4.631579,1.571429,3.761905,0.869674,1.788454
3,1626145,Tyus Jones,110,226,48.672566,81,163,49.693252,73,70,1.506849,3.095890,1.157143,2.328571,0.767319,-1.020685
4,1629636,Darius Garland,107,210,50.952381,93,212,43.867925,68,54,1.573529,3.088235,1.722222,3.925926,-0.837691,7.084456
5,203468,CJ McCollum,107,206,51.941748,62,128,48.437500,62,47,1.725806,3.322581,1.319149,2.723404,0.599176,3.504248
6,1630163,LaMelo Ball,93,203,45.812808,46,100,46.000000,75,51,1.240000,2.706667,0.901961,1.960784,0.745882,-0.187192
7,1630169,Tyrese Haliburton,98,202,48.514851,70,139,50.359712,77,58,1.272727,2.623377,1.206897,2.396552,0.226825,-1.844861
8,202704,Reggie Jackson,87,184,47.282609,30,80,37.500000,75,67,1.160000,2.453333,0.447761,1.194030,1.259303,9.782609
9,1630178,Tyrese Maxey,89,184,48.369565,49,102,48.039216,75,61,1.186667,2.453333,0.803279,1.672131,0.781202,0.330350


In [50]:
merged_target.loc[merged_target.SHOT_ATTEMPTED_FLAG_CUR > 10].sort_values(by='SHOT_PCT_CUR', ascending=False)

,PLAYER_ID,PLAYER_NAME,SHOT_MADE_FLAG_CUR,SHOT_ATTEMPTED_FLAG_CUR,SHOT_PCT_CUR,SHOT_MADE_FLAG_LAST,SHOT_ATTEMPTED_FLAG_LAST,SHOT_PCT_LAST,GP_CUR,GP_LAST,SHOT_MADE_FLAG_CUR_G,SHOT_ATTEMPTED_FLAG_CUR_G,SHOT_MADE_FLAG_LAST_G,SHOT_ATTEMPTED_FLAG_LAST_G,SHOT_ATTEMPTED_DIFF,SHOT_PCT_DIFF
20,203999,Nikola Jokic,97,141,68.794326,67,135,49.629630,74,72,1.310811,1.905405,0.930556,1.875000,0.030405,19.164697
87,1628389,Bam Adebayo,38,58,65.517241,41,78,52.564103,56,64,0.678571,1.035714,0.640625,1.218750,-0.183036,12.953139
85,1628969,Mikal Bridges,37,58,63.793103,5,15,33.333333,82,72,0.451220,0.707317,0.069444,0.208333,0.498984,30.459770
79,1626158,Richaun Holmes,38,62,61.290323,88,144,61.111111,45,61,0.844444,1.377778,1.442623,2.360656,-0.982878,0.179211
38,1628392,Isaiah Hartenstein,55,93,59.139785,16,28,57.142857,68,46,0.808824,1.367647,0.347826,0.608696,0.758951,1.996928
75,101108,Chris Paul,37,63,58.730159,25,49,51.020408,65,70,0.569231,0.969231,0.357143,0.700000,0.269231,7.709751
16,1628973,Jalen Brunson,88,150,58.666667,48,100,48.000000,79,68,1.113924,1.898734,0.705882,1.470588,0.428146,10.666667
37,1629634,Brandon Clarke,54,93,58.064516,39,83,46.987952,64,59,0.843750,1.453125,0.661017,1.406780,0.046345,11.076564
58,202696,Nikola Vucevic,41,72,56.944444,23,47,48.936170,73,70,0.561644,0.986301,0.328571,0.671429,0.314873,8.008274
83,1629028,Deandre Ayton,33,59,55.932203,5,10,50.000000,58,69,0.568966,1.017241,0.072464,0.144928,0.872314,5.932203


### Fadeaways

In [35]:
target_shots_only = current_df.loc[current_df.ACTION_TYPE.str.contains('Fadeaway')]

target_shots_agg = target_shots_only.groupby(['PLAYER_ID', 'PLAYER_NAME']).agg(
    {
        'SHOT_MADE_FLAG': 'sum',
        'SHOT_ATTEMPTED_FLAG': 'sum'
    }
).reset_index()

target_shots_agg = target_shots_agg.sort_values(by='SHOT_ATTEMPTED_FLAG', ascending=False)

target_shots_agg.loc[:, 'SHOT_PCT_CUR'] = target_shots_agg.SHOT_MADE_FLAG / target_shots_agg.SHOT_ATTEMPTED_FLAG * 100

In [36]:
target_last = last_df.loc[last_df.ACTION_TYPE.str.contains('Fadeaway')]
target_last_agg = target_last.groupby(['PLAYER_ID', 'PLAYER_NAME']).agg(
    {
        'SHOT_MADE_FLAG': 'sum',
        'SHOT_ATTEMPTED_FLAG': 'sum'
    }
).reset_index()

target_last_agg = target_last_agg.sort_values(by='SHOT_ATTEMPTED_FLAG', ascending=False)

target_last_agg.loc[:, 'SHOT_PCT_LAST'] = target_last_agg.SHOT_MADE_FLAG / target_last_agg.SHOT_ATTEMPTED_FLAG * 100

In [37]:
merged_target = pd.merge(left=target_shots_agg, right=target_last_agg, on=('PLAYER_ID', 'PLAYER_NAME'), suffixes=('_CUR', '_LAST'))

merged_target = merged_target.sort_values(by='SHOT_ATTEMPTED_FLAG_CUR', ascending=False)

names_games = player_stats.loc[:, ['PLAYER_ID', 'PLAYER_NAME', 'GP']]
names_games_last = player_stats_last.loc[:, ['PLAYER_ID', 'PLAYER_NAME', 'GP']]

merged_target = pd.merge(left=merged_target, right=names_games, on=['PLAYER_ID', 'PLAYER_NAME'])

merged_target = pd.merge(left=merged_target, right=names_games_last, on=['PLAYER_ID', 'PLAYER_NAME'], suffixes=('_CUR', '_LAST'))

merged_target.loc[:, 'SHOT_MADE_FLAG_CUR_G'] = merged_target.SHOT_MADE_FLAG_CUR / merged_target.GP_CUR
merged_target.loc[:, 'SHOT_ATTEMPTED_FLAG_CUR_G'] = merged_target.SHOT_ATTEMPTED_FLAG_CUR / merged_target.GP_CUR
merged_target.loc[:, 'SHOT_MADE_FLAG_LAST_G'] = merged_target.SHOT_MADE_FLAG_LAST / merged_target.GP_LAST
merged_target.loc[:, 'SHOT_ATTEMPTED_FLAG_LAST_G'] = merged_target.SHOT_ATTEMPTED_FLAG_LAST / merged_target.GP_LAST

merged_target = merged_target.loc[(merged_target.GP_CUR > 15) & (merged_target.GP_LAST > 15) & (merged_target.SHOT_ATTEMPTED_FLAG_CUR > 50)]

merged_target.loc[:, 'SHOT_ATTEMPTED_DIFF'] = merged_target.SHOT_ATTEMPTED_FLAG_CUR_G - merged_target.SHOT_ATTEMPTED_FLAG_LAST_G

merged_target.loc[:, 'SHOT_PCT_DIFF'] = merged_target.SHOT_PCT_CUR - merged_target.SHOT_PCT_LAST

In [38]:
merged_target.loc[merged_target.SHOT_ATTEMPTED_FLAG_CUR > 1].sort_values(by='SHOT_ATTEMPTED_FLAG_CUR', ascending=False)

,PLAYER_ID,PLAYER_NAME,SHOT_MADE_FLAG_CUR,SHOT_ATTEMPTED_FLAG_CUR,SHOT_PCT_CUR,SHOT_MADE_FLAG_LAST,SHOT_ATTEMPTED_FLAG_LAST,SHOT_PCT_LAST,GP_CUR,GP_LAST,SHOT_MADE_FLAG_CUR_G,SHOT_ATTEMPTED_FLAG_CUR_G,SHOT_MADE_FLAG_LAST_G,SHOT_ATTEMPTED_FLAG_LAST_G,SHOT_ATTEMPTED_DIFF,SHOT_PCT_DIFF
0,203954,Joel Embiid,82,220,37.272727,67,137,48.905109,68,51,1.205882,3.235294,1.313725,2.686275,0.549020,-11.632382
1,201942,DeMar DeRozan,99,215,46.046512,36,60,60.000000,76,61,1.302632,2.828947,0.590164,0.983607,1.845341,-13.953488
2,201142,Kevin Durant,81,159,50.943396,29,54,53.703704,55,35,1.472727,2.890909,0.828571,1.542857,1.348052,-2.760307
3,1629029,Luka Doncic,62,153,40.522876,63,110,57.272727,65,66,0.953846,2.353846,0.954545,1.666667,0.687179,-16.749851
4,203944,Julius Randle,48,150,32.000000,79,176,44.886364,72,71,0.666667,2.083333,1.112676,2.478873,-0.395540,-12.886364
5,2544,LeBron James,62,140,44.285714,46,103,44.660194,56,45,1.107143,2.500000,1.022222,2.288889,0.211111,-0.374480
6,1628369,Jayson Tatum,48,127,37.795276,47,151,31.125828,76,64,0.631579,1.671053,0.734375,2.359375,-0.688322,6.669448
7,203897,Zach LaVine,54,125,43.200000,22,41,53.658537,67,58,0.805970,1.865672,0.379310,0.706897,1.158775,-10.458537
8,203114,Khris Middleton,48,123,39.024390,51,101,50.495050,66,68,0.727273,1.863636,0.750000,1.485294,0.378342,-11.470659
9,1626164,Devin Booker,49,107,45.794393,51,107,47.663551,68,67,0.720588,1.573529,0.761194,1.597015,-0.023486,-1.869159
